<a href="https://colab.research.google.com/github/stravo1/Colab-Notebooks/blob/main/Langchain_OpenRouter_Integration_FunctionCall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-dotenv openai langchain_core

In [2]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

True

In [3]:
from typing import Any, List, Mapping, Optional, Union, Sequence, Tuple, Dict, Iterator

from openai import OpenAI
from os import getenv

from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain_core.prompt_values import PromptValue
from langchain_core.messages.base import BaseMessage
from langchain_core.runnables.config import RunnableConfig

In [4]:
custom_functions = [
    {
        'name': 'extract_student_info',
        'description': 'Get the student information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'major': {
                    'type': 'string',
                    'description': 'Major subject.'
                },
                'school': {
                    'type': 'string',
                    'description': 'The university name.'
                },
                'grades': {
                    'type': 'integer',
                    'description': 'GPA of the student.'
                },
                'club': {
                    'type': 'string',
                    'description': 'School club for ONLY extracurricular activities. NOT AN ARRAY, CHOOSE ONE.'
                }

            }
        }
    },
    {
        'name': 'extract_school_info',
        'description': 'Get the school information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the school.'
                },
                'ranking': {
                    'type': 'integer',
                    'description': 'QS world ranking of the school.'
                },
                'country': {
                    'type': 'string',
                    'description': 'Country of the school.'
                },
                'total': {
                    'type': 'integer',
                    'description': 'Total number of students in the school',
                },
                'u_grads': {
                    'type': 'integer',
                    'description': 'Total number of undergraduate students in the school',
                },
                'grads': {
                    'type': 'integer',
                    'description': 'Total number of graduate students in the school',
                },
                'summary': {
                    'type': 'string',
                    'description': 'The summary for the school in a few words',
                },
                'keywords': {
                    'type': 'array',
                    'items': {
                        'type': 'string'
                    },
                    'description': 'The one-word keywords associated with the school, provide at least 5 of them.',
                }
            }
        }
    }
]

tools = [
    {
        "type": "function",
        "function": custom_functions[0]
    },
    {
        "type": "function",
        "function": custom_functions[1]
    }
]

school_1_description = "Stanford University is a private research university located in Stanford, California, United States. It was founded in 1885 by Leland Stanford and his wife, Jane Stanford, in memory of their only child, Leland Stanford Jr. The university is ranked #5 in the world by QS World University Rankings. It has over 17,000 students, including about 7,600 undergraduates and 9,500 graduates. "
student_1_description = "David Nguyen is a sophomore majoring in computer science at Stanford University. He is Asian American and has a 3.8 GPA. David is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after graduating."
student_2_description="Ravi Patel is a sophomore majoring in computer science at the University of Michigan. He is South Asian Indian American and has a 3.7 GPA. Ravi is an active member of the university's Chess Club and the South Asian Student Association. He hopes to pursue a career in software engineering after graduating."

In [5]:
# create an OpenAI API client
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=getenv("API_KEY"),
)

# more models at https://openrouter.ai/models
model = "mistralai/mistral-7b-instruct:nitro"

system_prompt = {
      "role": "system",
      "content": r"You are an AI assistant who has access to various functions and depending on the request you need to call the function with the functoin name and proper arguments in the JSON format {'function_name':'...', 'arguments': {...}} strictly following the types. No extra content needed.",
}

model_config = {
    "model" : model,
    "tools" : tools,
    "temperature" : 0,
    "tool_choice" : "auto",
    "response_format" : {"type": "json_object"},
    "max_tokens": 1000,
}

# response is delivered at once instead of stremaing
def get_completed_response(prompt):
    completion = client.chat.completions.create(
        **model_config,
        messages = [
            system_prompt,
            {
              "role": "user",
              "content": prompt,
            }
          ],
    )
    return completion.choices[0].message.content

# response is streamed
def get_stream(prompt):
    stream = client.chat.completions.create(
        **model_config,
        stream = True,
        messages = [
            system_prompt,
            {
                "role": "user",
                "content": prompt,
            }
        ]
    )
    return stream;

# inherit the LLM class from langchain
class OpenRouter_FunctionCalling(LLM):

    @property
    def _llm_type(self) -> str:
        return "open-router"

    # implementing the _call function is mandatory
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        return "gg"

    def invoke(
        self,
        input: str,
        config: Optional[RunnableConfig] = None, *,
        stop: Optional[List[str]] = None,
        **kwargs: Any
    ) -> str:
        return get_completed_response(input)

    def stream(
        self,
        input: str,
        config: Optional[RunnableConfig] = None, *,
        stop: Optional[List[str]] = None,
        **kwargs: Any
        ) -> Iterator[str]:
        return get_stream(input)


In [13]:
import json

llm = OpenRouter_FunctionCalling()

print(json.loads(llm.invoke(school_1_description)))
print(json.loads(llm.invoke(student_2_description)))

{'function_name': 'extract_school_info', 'arguments': {'name': 'Stanford University', 'ranking': '5', 'country': 'United States', 'total': '17000', 'u_grads': '7600', 'grads': '9500', 'summary': 'Private research university', 'keywords': ['Stanford', 'California', 'Leland Stanford', 'Jane Stanford', 'Research', 'University']}}
{'function_name': 'extract_student_info', 'arguments': {'name': 'Ravi Patel', 'major': 'computer science', 'school': 'University of Michigan', 'grades': '3.7', 'club': 'Chess Club'}}
